# 05 Tooling | 01 Semantic Kernel | 02 Vector Store

## Azure Environment

Necessary parameter are imported from [./Configuration/application.env]. Check [Create Environment](../../01_CreateEnvironment/01_Environment.ipynb) to setup the necessary demo environment.

This notebooks highlights how Semantic Kernel simplifies the process of storing and retrieving vector data. 

## Step 1: Create Semantic Kernel

MS Semantic Kernel is a tool or library developed by Microsoft to abstract away communication with LLMs and other services like vector DBs. The nuget package Microsoft.SemanticKernel provides developers with access to this powerful tool, allowing them to integrate advanced language processing capabilities into their applications effortlessly.


In [1]:
#pragma warning disable SKEXP0001, SKEXP0010 

#r "nuget: Microsoft.SemanticKernel, 1.48.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.48.0-preview"
#r "nuget: Microsoft.Extensions.VectorData.Abstractions, 9.0.0-preview.1.25229.1"
#r "nuget: DotNetEnv, 3.1.1"

using System.IO;
using System.ClientModel;
using System.ComponentModel;
using DotNetEnv;
using Microsoft.Extensions.VectorData;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Connectors.InMemory;

//configuration file is created during environment creation
static string _configurationFile = @"../../Configuration/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("WS_EMBEDDING_DEPLOYMENTNAME") ?? "WS_EMBEDDING_DEPLOYMENTNAME not found";
string assetFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";  


Kernel kernel = Kernel.CreateBuilder()
    .AddAzureOpenAITextEmbeddingGeneration(
        apiKey: oAiApiKey, 
        endpoint: oAiEndpoint, 
        deploymentName: embeddingDeploymentName
    )
.Build();
Console.WriteLine($"Semantic Kernel created...");


Installed Packages DotNetEnv, 3.1.1 Microsoft.Extensions.VectorData.Abstractions, 9.0.0-preview.1.25229.1 Microsoft.SemanticKernel, 1.48.0 Microsoft.SemanticKernel.Connectors.InMemory, 1.48.0-preview

Semantic Kernel created...


## Step 2: Define Storage Model

Semantic Kernel offers an abstraction of creating embeddings and storing/querying them in connected vector databases.

A data model can be used to store and query data.


In [2]:
public sealed class DataStorageModel
{
    #pragma warning disable SKEXP0001
    [VectorStoreRecordKey]
    [TextSearchResultName]
    public Guid Key { get; init; }

    [VectorStoreRecordData]
    [TextSearchResultValue]
    public string Text { get; init; } = "Empty Record - No Data"; 

    [VectorStoreRecordVector(3072)]
    public ReadOnlyMemory<float> Embedding { get; init; }
    
    [TextSearchResultLink]
    public string Link {get; set;} = "";
    #pragma warning restore SKEXP0001
}

Console.WriteLine($"DataStorageModel defined...");

DataStorageModel defined...


## Step 3: Create Embeddings

Information provided in text files will be embedded for further storage and querying.

In [3]:
using System.IO; 

#pragma warning disable SKEXP0001, SKEXP0010 

ITextEmbeddingGenerationService textEmbeddingGenerationService = kernel.GetRequiredService<ITextEmbeddingGenerationService>();

List<string> knowledge = new List<string>(); 
knowledge.Add(await File.ReadAllTextAsync(Path.Combine(assetFolder, "Embedding", "WikiAks.txt")));
knowledge.Add(await File.ReadAllTextAsync(Path.Combine(assetFolder, "Embedding", "WikiSuperBowl2024.txt")));
knowledge.Add("The Kansas City Chiefs won the Super Bowl 2024!");

IList<ReadOnlyMemory<float>> vectorData = 
    await textEmbeddingGenerationService.GenerateEmbeddingsAsync(knowledge);

Console.WriteLine($"Embeddings generated...")

Embeddings generated...


## Step 4: Create Vector Store

The code cell adds specific information or knowledge to a vector store abstraction. In this sample an instance of `InMemoryVectorStore()` is used and knowledge `List<string> knowledge` is transferred to the StorageModel and uploaded to the vector store.

In [4]:
using System.IO; 

#pragma warning disable SKEXP0001, SKEXP0010 

InMemoryVectorStore inMemoryStore = new InMemoryVectorStore();
string collectionName = "knowledgebase";

IVectorStoreRecordCollection<Guid, DataStorageModel> recordCollection = 
    inMemoryStore.GetCollection<Guid, DataStorageModel>(collectionName);
await recordCollection.CreateCollectionIfNotExistsAsync();

List<DataStorageModel> dataStorageRecords = knowledge
    .Select((text, index) => 
    {
        var key = Guid.NewGuid();
        return new DataStorageModel
        {
            Key = key,
            Text = text,
            Embedding = vectorData[index],
            Link = $"data:citation/text;{key}"
        };
    })
    .ToList();

Console.WriteLine($"DataStorageRecords created: ");
foreach (DataStorageModel dataStorageModel in dataStorageRecords)
{
    await recordCollection.UpsertAsync(dataStorageModel);
    Console.WriteLine($"\t{dataStorageModel.Key}");
}

Console.WriteLine($"All records stored in vector store..."); 

DataStorageRecords created: 
	192a507e-4744-4e03-a771-14be13644ace
	51eae057-6192-4607-a523-2146d86312df
	14fa68d6-54c7-4948-b64c-a6d3dec6d087
All records stored in vector store...


## Step 4: Query Memory Store

The code cell uses the `VectorizedSearchAsync()` function to query the vector store. 

In [5]:
using Microsoft.Extensions.VectorData; 

ReadOnlyMemory<float> searchVector = await textEmbeddingGenerationService.GenerateEmbeddingAsync(
    "What is the Super Sports Championship");

IAsyncEnumerable<VectorSearchResult<DataStorageModel>> searchResults = recordCollection.SearchEmbeddingAsync<ReadOnlyMemory<float>>(
    vector: searchVector, 
    top: 2
);

await foreach (VectorSearchResult<DataStorageModel> result in searchResults)
{
        
    Console.WriteLine($"Score: {result.Score} \nText: {result.Record.Text.Substring(0, 10)}... \nCitation: {result.Record.Link} \n");
}

Console.WriteLine($"Search completed...");

Score: 0.7757171988487244 
Text: Super Bowl... 
Citation: data:citation/text;51eae057-6192-4607-a523-2146d86312df 

Score: 0.7739335894584656 
Text: The Kansas... 
Citation: data:citation/text;14fa68d6-54c7-4948-b64c-a6d3dec6d087 

Search completed...
